In [1]:
!pip install yfinance==0.2.31

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peewee: filename=peewee-3.17.0-py3-none-any.whl size=135720 sha256=ec5ce290483ac6b610bc96241247857f6c2e4cb86038e57294737c1ada8ae9ae
  Stored in directory: /root/.cache/pip/wheels/77/6b/dc/722aee2a5d46a021ab67564c9d3d5536c3bd9577fcdc3a5aeb
Successfully built peewee
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Not uninst

In [2]:
import time

import pandas as pd
import numpy as np

import yfinance as yf

import matplotlib.pyplot as plt
import altair as alt

In [3]:
# Parameters

stock_ticker = "INTC"


In [4]:
# Fetch/Make Calls to y-finance

stock = yf.Ticker(stock_ticker)

dividends = stock.dividends.loc['2003-01-01':].to_frame().reset_index()
stock_price_history = stock.history(period="240mo").reset_index() # 20 years



In [5]:
## Pre-processing:
dividends_by_year = dividends.groupby(dividends.Date.dt.year)['Dividends'].sum()
stock_price_by_year = stock_price_history.groupby(stock_price_history.Date.dt.year)['Close'].mean()

dividend_yield_by_year =  dividends_by_year / stock_price_by_year * 100
dividend_yield_by_year.rename('Dividend Yield (%)', inplace=True)

df=pd.concat([dividends_by_year,stock_price_by_year,dividend_yield_by_year],axis=1)

# Calculate Dividend Growth pct:

growth_pcts = []
for i in range(0,len(df.Dividends)-1):

    growth_pct = round(df.Dividends.iloc[i+1]/df.Dividends.iloc[i] - 1,3)*100

    growth_pcts.append(growth_pct)

growth_pcts.insert(0, np.nan)
df['Dividend Growth % year-over-year'] = growth_pcts
df.reset_index(inplace = True)

# remove current year because incomplete data (unless able to incorporate Wall Street estimates)
df = df.iloc[:-1,:]

## Merging

combined = pd.merge(dividends.assign(grouper=dividends['Date'].dt.to_period('Y')),
               stock_price_history.assign(grouper=stock_price_history['Date'].dt.to_period('Y')),
               how='left', on='grouper')

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [6]:
alt.Chart(dividends).mark_line().encode(
    x= "Date",
    y='Dividends:Q'
).properties(
    title = f"Last 10 Years - Dividend Rate - {stock_ticker}"
)

alt.Chart(...)

In [7]:
df.head()

,Date,Dividends,Close,Dividend Yield (%),Dividend Growth % year-over-year
0,2003,0.080,18.798018,0.425577,NaN
1,2004,0.160,14.973603,1.068547,100.0
2,2005,0.320,14.755819,2.168636,100.0
3,2006,0.400,12.013417,3.329611,25.0
4,2007,0.452,14.394735,3.140037,13.0


In [8]:
dividend_yield_line_chart = alt.Chart(df).mark_line(color = 'black').encode(
    x= alt.X("Date:O"),
    y= alt.Y('Dividend Yield (%):Q', title = 'Dividend Yield (%')
).properties(
    title = f"Dividend Analysis: {stock_ticker}"
)

dividend_growth_bar_chart = alt.Chart(df).mark_bar(opacity=0.7, color='#57A44C').encode(
    x= alt.X('Date:O'),
    y=alt.Y('Dividend Growth % year-over-year:Q', title = f'{stock_ticker} Dividend Growth, Year-over-Year(%)', 
        ),

)

In [9]:
alt.layer(dividend_yield_line_chart, dividend_growth_bar_chart ).resolve_scale(
    y='independent'
).configure_axisLeft(titleColor='black').configure_axisRight(titleColor='#57A44C')

alt.LayerChart(...)

In [10]:
div_growth_20_years = np.mean(df['Dividend Growth % year-over-year'])
yield_20_years = np.mean(df['Dividend Yield (%)'])

div_growth_10_years = np.mean(df['Dividend Growth % year-over-year'].iloc[9:])
yield_10_years = np.mean(df['Dividend Yield (%)'].iloc[9:])

div_growth_5_years = np.mean(df['Dividend Growth % year-over-year'].iloc[-5:])
yield_5_years = np.mean(df['Dividend Yield (%)'].iloc[-5:])

In [11]:
df.Date.iloc[-5]

2018

In [12]:
def div_average_text():
    print("Averages           | Div Growth %  | Yield" )
    if len(df.Date) > 19:
        print(f"{df.Date[0]}-{df.Date[19]} (20 yrs) | {div_growth_20_years:.2f} | {yield_20_years:.2f}")
        print(f"{df.Date[9]}-{df.Date[19]} (10 yrs) | {div_growth_10_years:.2f} | {yield_10_years:.2f}")
        print(f"{df.Date[15]}-{df.Date[19]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")

    elif len(df.Date) > 9:
        print(f"{df.Date.iloc[-10]}-{df.Date.iloc[-1]} (10 yrs) | {div_growth_10_years:.2f} | {yield_10_years:.2f}")
        print(f"{df.Date.iloc[-5]}-{df.Date.iloc[-1]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")

    elif len(df.Date) > 4:
        print(f"{df.Date.iloc[-5]}-{df.Date[-1]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")


    else:
        print('Historical Data less than 5 years')    


In [13]:
div_average_text()

Averages           | Div Growth %  | Yield
2003-2022 (20 yrs) | 19.10 | 3.45
2012-2022 (10 yrs) | 5.90 | 3.62
2018-2022 (5 yrs)  | 6.28 | 2.97


In [14]:
df.head(20)

,Date,Dividends,Close,Dividend Yield (%),Dividend Growth % year-over-year
0,2003,0.080,18.798018,0.425577,NaN
1,2004,0.160,14.973603,1.068547,100.0
2,2005,0.320,14.755819,2.168636,100.0
3,2006,0.400,12.013417,3.329611,25.0
4,2007,0.452,14.394735,3.140037,13.0
5,2008,0.548,12.524702,4.375354,21.2
6,2009,0.560,11.112000,5.039597,2.2
7,2010,0.632,13.898587,4.547225,12.9
8,2011,0.782,15.250821,5.127593,23.7
9,2012,0.870,17.838418,4.877114,11.3


## Dividend Drill Return Model

In [15]:
stock.info['returnOnEquity']

-0.00911

In [16]:
# See chapter 7 in Dividend Playbook

date = df.iloc[-1].Date.astype('int')

dividend_rate = df.iloc[-1].Dividends
share_price = df.iloc[-1].Close
current_yield = dividend_rate / share_price 

# 2022 Net Income
net_income = stock.income_stmt.loc['Net Income'][0]

# number of shares out there.
shares_outstanding = stock.info['sharesOutstanding']

# Earnings Per Share (accounts for dividend)
EPS = net_income / shares_outstanding


# Return on Equity
ROE = stock.info['returnOnEquity']

# Payout Ratio
try:
    payout_ratio = stock.info['payoutRatio']

except:
    payout_ratio = None

# Sustainable Growth Rate (not fool-proofed)
#SGR = (1-payout_ratio) * ROE

# Free Growth Estimate
FGE = 0.00

# Growth Rate (requires some thought)
GR_estimate = 0.05

cost_of_growth = GR_estimate / ROE * EPS

funding_gap = EPS  - dividend_rate - cost_of_growth

share_change = funding_gap / share_price

total_dividend_growth = GR_estimate + share_change

projected_total_return = total_dividend_growth + current_yield

In [17]:
print(date)
div_average_text()

2022
Averages           | Div Growth %  | Yield
2003-2022 (20 yrs) | 19.10 | 3.45
2012-2022 (10 yrs) | 5.90 | 3.62
2018-2022 (5 yrs)  | 6.28 | 2.97


In [18]:
print(f'{date} - {stock_ticker}')

print("---------------------------------------")

print(f"Dividend Rate ($): {dividend_rate:.2f}")
print(f"Divided by Share Price ($) {share_price:.2f}")
print(f"Current Yield (%): {current_yield*100:.2f}")

print("---------------------------------------")

print(f"Core Growth Estimate (%): {GR_estimate * 100:.2f}")
print(f"Divided by: Return on Equity (%): {ROE * 100:.2f}")
print(f"Multipled by: Earnings per Share ($): {EPS:.2f}")
print(f"Cost of Growth ($): {cost_of_growth:.2f}")

print("---------------------------------------")
print(f"Earnings per Share ($): {EPS:.2f}")
print(f"Minus: Dividend: {dividend_rate:.2f}")
print(f"Minute: Cost of Growth ($): {cost_of_growth:.2f}")
print(f"Funding Gap ($): {funding_gap:.2f}")

print("---------------------------------------")
print(f"Funding Gap ($): {funding_gap:.2f}")
print(f"Divided by: Share Price ($): {EPS:.2f}")
print(f"Share Change (%): {share_change *100:.2f}")

print("---------------------------------------")
print(f"Core Growth Estimate (%): {GR_estimate*100:.2f}")
print(f"Plus: Share Change (%): {share_change*100:.2f}")
print(f"Total Dividend Growth (%): {total_dividend_growth*100:.2f}")


print("---------------------------------------")
print(f"Plus: Dividend Yield (%): {current_yield*100:.2f}")
print(f"Projected Total Return (%): {projected_total_return*100:.2f}")



2022 - INTC
---------------------------------------
Dividend Rate ($): 1.46
Divided by Share Price ($) 36.98
Current Yield (%): 3.95
---------------------------------------
Core Growth Estimate (%): 5.00
Divided by: Return on Equity (%): -0.91
Multipled by: Earnings per Share ($): 1.91
Cost of Growth ($): -10.50
---------------------------------------
Earnings per Share ($): 1.91
Minus: Dividend: 1.46
Minute: Cost of Growth ($): -10.50
Funding Gap ($): 10.96
---------------------------------------
Funding Gap ($): 10.96
Divided by: Share Price ($): 1.91
Share Change (%): 29.63
---------------------------------------
Core Growth Estimate (%): 5.00
Plus: Share Change (%): 29.63
Total Dividend Growth (%): 34.63
---------------------------------------
Plus: Dividend Yield (%): 3.95
Projected Total Return (%): 38.58


## Margin of Safety

The basic margin of safety principle is to pay less for a stock than it is worth.



<img src="image-20231013-103415.png" width="" align="" />

In [19]:
print(f"For {date} {stock_ticker}: \nDividend Yield (%): {current_yield*100:.2f}\nDividend Growth (%): {total_dividend_growth*100:.2f}")



For 2022 INTC: 
Dividend Yield (%): 3.95
Dividend Growth (%): 34.63


In [26]:
"""
required_return - the demand of return we want from our investment. See table above to make a decision
"""
hurdle_return = .09

hurdle_price = dividend_rate / (hurdle_return - total_dividend_growth)

projected_dividend_yield = dividend_rate / hurdle_price

ratio = hurdle_price / share_price

In [27]:
print(hurdle_price)
print(projected_dividend_yield )
print(ratio)

-5.6966642678749055
-0.25629033612413343
-0.15405722560189544


### Research/References

1. Documentation
    - https://github.com/ranaroussi/yfinance
2. Articles
    - The Ultimate Dividend Playbook: Income, Insight and Independence for Today's Investor, Peters, Josh
    - https://dividendgrowthinvestingandretirement.com/estimate-dividend-growth-total-returns-using-josh-peters-dividend-drill-return-model-example-spreadsheet/#:~:text=Guessing%20a%20reasonable%20future%20dividend%20growth%20rate%20is%20hard%2C%20but,dividend%20growth%20and%20total%20returns.
    - Negative cost of growth: https://www.investopedia.com/terms/n/negative-growth.asp#:~:text=An%20economy%20with%20negative%20growth,of%20a%20recession%20or%20depression.
3. References
    - https://www.nasdaq.com/market-activity/quotes/dividend-history
4. Questions:
    - what happens if the calculated hurdle price is in the negative? that can't be a good thing? it means, the growth I am looking for is not sustainable. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67b8993f-5b2d-4ce3-8c93-aa21b89c512f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>